# ZNE with indirect-control VQE
---

This experiment is:

- Qubit: 7, layer: 30
- System has noise-free time evolution.
- Target observable: Transverse field Ising Hamiltonian.
- Ansatz time-evolution: XY, Ising, and Heisenberg Hamiltonians.

## Import and setup

In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate

In [2]:
# Set the default font to be LaTeX compliant.
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Computer Modern Roman"
plt.rcParams["text.usetex"] = True

# Set a global font size.
font_size = 11
plt.rcParams["font.size"] = font_size
plt.rcParams["axes.labelsize"] = font_size
plt.rcParams["xtick.labelsize"] = font_size
plt.rcParams["ytick.labelsize"] = font_size
plt.rcParams["legend.fontsize"] = 10

## (2) Loading the experimental data from JSON files

The raw experimenal data are availabe at `data\` folder

In [14]:
experimental_data = {
    "xy_data": None,
    "ising_data": None,
    "heisenberg_data": None,
}

# Define the paths to the subfolders
base_path = "data"
subfolders = ["xy", "ising", "heisenberg"]

# Initialize a dictionary to store the experimental data
experimental_data = {}

# Load JSON data from each subfolder
for subfolder in subfolders:
    folder_path = os.path.join(base_path, subfolder)
    json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
    
    # Load all JSON files in the subfolder
    data = []
    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)
        with open(file_path, "r") as f:
            data.append(json.load(f))
    
    # Store the data in the dictionary
    experimental_data[f"{subfolder}_data"] = data


## (3) Extracting data

Now lets extract some unformations.

### (A) The exact minimum eigen value

The exact minimum eigen value of the target observable (i.e. tranverse field Ising Hamiltonian for a 7-qubit system) is $-8.566772233505624$. This value is recored in `file_name_prefix_ZNE.json` files.

In [16]:
exact_sol: float = -8.566772233505624
print(f"Exact solution: {exact_sol}")

Exact solution: -8.566772233505624
